# Анализ пресс-релизов Банка России

**Описание исследования:**

Мы получили тексты пресс-релизов с сайта ЦБ, и теперь хотим увидеть взаимосвязь этих текстов с решением, по ключевой ставке.

**Цель исследования:**

Получить знания, о зависимости текстовых данных пресс-релизов и решениями по ключевой ставки и использовать их в построении модели предсказания ключевой ставки.

**План работы:**
1) Загрузка и предобработка данных.
2) Создание целевых переменных.
3) Провести анализ текстов: расчет средней, минимальной и максимальной длины текстов (визуализация
распределения), частотный анализ слов и n-грамм, извлечение биграмм и триграмм, статистики по
символам и словам, распределение частей речи.
4) Изучить связь текстовых признаков с целевой переменной.
5) Визуализировать данные:
    - распределения классов;
    - облако наиболее частотных слов и биграмм для выявления ключевых тем (Word Clouds);
    - методы снижения размерности (например, t-SNE) для визуализации текстовых представлений.
6) Провести первичную предобработку теста:
    - оценить и обработать пропуски, дубликаты; очистить текст от шумовых данных: ненужных символов, HTML-тегов,
специальных символов;
    - преобразовать регистр, исключить стоп-слова, провести токенизацию (разбить на слова, предложения, символы), лемматизацию или стемминг.
7) Оформить вывод о структуре данных, их особенностях и потенциальных проблемах на основе
проведенного разведочного анализа данных.

**Описание данных:**

Данные получены путем скрапинга с официального сайти Центрального банка.

- `date` - дата опубликования пресс-релиза;
- `link` - ссылка на пресс-релиз;
- `title` - заголовок пресс-релиза;
- `release` - текст пресс-релиза.

In [49]:
# импортируем библиотеки
import re

import pandas as pd
import locale
from datetime import datetime

## Загрузка и предобработка данных.

Загрузим данные и взглянем на первые и последние 5 строчек.

In [2]:
df = pd.read_csv('../data/raw-cbr-press-releases.csv')
df

,date,link,title,release
0,13 сентября 2024 г.,/press/pr/?file=13092024_133000Key.htm,Банк России принял решение повысить ключевую с...,\n \nСовет директоров Банка России 13 сентября...
1,26 июля 2024 г.,/press/pr/?file=26072024_133000Key.htm,Банк России принял решение повысить ключевую с...,\n \nСовет директоров Банка России 26 июля 202...
2,7 июня 2024 г.,/press/pr/?file=07062024_133000Key.htm,Банк России принял решение сохранить ключевую ...,\n \nСовет директоров Банка России 7 июня 2024...
3,26 апреля 2024 г.,/press/pr/?file=26042024_133000key.htm,Банк России принял решение сохранить ключевую ...,\n \nСовет директоров Банка России 26 апреля 2...
4,22 марта 2024 г.,/press/pr/?file=22032024_133000key.htm,Банк России принял решение сохранить ключевую ...,\n \nСовет директоров Банка России 22 марта 20...
...,...,...,...,...
95,13 декабря 2013 г.,/press/pr/?file=131213_133004stavka_table.htm,О ключевой ставке Банка России,\nСовет директоров Банка России 13 декабря 201...
96,8 ноября 2013 г.,/press/pr/?file=131108_133008stavka_+table.htm,О ключевой ставке Банка России,\nСовет директоров Банка России 8 ноября 2013 ...
97,14 октября 2013 г.,/press/pr/?file=131014_133001refi_rate_tab.htm,О ключевой ставке Банка России,"\nПресс-служба Банка России сообщает, что Сове..."
98,13 сентября 2013 г.,/press/pr/?file=130913_1350427l.htm,О системе процентных инструментов денежно-кред...,\n О системе процентных инструментов денежно...


К ссылкам стоит добавить адрес главной страницы, тексты релизов стоит обработать и убрать лишние символы перевода строки. Дату надо привести к соответствующему типу данных. Посмотрим какие типы данных у наших переменных и количество пропущенных значений.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     100 non-null    object
 1   link     100 non-null    object
 2   title    100 non-null    object
 3   release  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


У нас 100 наблюдений в нашем датасете. Пропущенные значения отсутствуют. Дату, как уже упоминалось выше стоит превести к типу данных `date`.

In [4]:
locale.setlocale(
    category=locale.LC_ALL,
    locale="Russian"
)

result = datetime.strptime('26 май 2022', '%d %b %Y')
result

def calc_date(value):
    day, month, year, _ = value.split()
    month = month[:3] if month != 'мая' else 'май'
    return datetime.strptime(' '.join([day, month, year]), '%d %b %Y')

df.date = df.date.map(calc_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     100 non-null    datetime64[ns]
 1   link     100 non-null    object        
 2   title    100 non-null    object        
 3   release  100 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.3+ KB


Теперь порядок с типами данных. Проверим данные на дубликаты.

In [5]:
print(f'Количество дубликатов по дате и названию - {df.duplicated(subset=["date", "title"]).sum()}')

Количество дубликатов по дате и названию - 0


Посмотрим есть ли у нас релизы опубликованные в один день.

In [6]:
print(f'Количество дубликатов по дате - {df.duplicated(subset=["date"]).sum()}')

Количество дубликатов по дате - 2


Взглянем на эти релизы.

In [27]:
indexes = df[df.duplicated(subset=["date"], keep=False)].index

for index in indexes:
    release = df.loc[index]
    print(release.date)
    print('-' * 80, end=' ')
    for i in range(0, len(release.release), 80):
        
        print(release.release[i:i+80])

2015-09-11 00:00:00
-------------------------------------------------------------------------------- 
В целях оперативного информирования пользователей о макроэкономических тенденци
ях, складывающихся в сфере внешнеэкономической деятельности, Банк России вводит 
в практику ежемесячные сообщения об оценке ключевых агрегатов платежного баланса
 Российской Федерации. 
Данные будут размещаться в рубрике Комментарии Банка Рос
сии раздела Пресс-центр официального сайта Банка России.При использовании матери
ала ссылка на Пресс-службу Банка России обязательна.

2015-09-11 00:00:00
-------------------------------------------------------------------------------- 
Совет директоров Банка России 11 сентября 2015 года принял решение сохранить кл
ючевую ставку на уровне 11,00% годовых, учитывая увеличение инфляционных рисков 
при сохранении рисков существенного охлаждения экономики. В августе произошло зн
ачительное ухудшение внешнеэкономической конъюнктуры. Под воздействием курсовой 
динамики выросл

Мы можем заметить, что два релиза из четырех не связаны с решением банка россии по ключевой ставке. Удалим эти наблюдения из выборки.

In [29]:
df.drop_duplicates(subset=['date'], keep='last', inplace=True)

## Создание целевых переменных.

Создадим переменную `target_category`, которая будет равна 1 если ЦБ повысил ставку, 0 если ставка не изменилась и -1 если ставка снизилась.

In [32]:
def calc_target(row):
    if 'сохранить' in row['title'] or 'сохранил' in row['title']:
        return 0
    elif 'повысить' in row['title']:
        return 1
    elif 'снизить' in row['title']:
        return -1
    else:
        end = 150
        
        if ('О ключевой ставке Банка России' in row['title'] 
            or 'О процентных ставках по операциям Банка России' in row['title']):

            if 'сохранить' in row['release'][:end] or 'оставить' in row['release'][:end]:
                return 0
            elif 'повысить' in row['release'][:end]:
                return 1
            elif 'снизить' in row['release'][:end]:
                return -1

In [33]:
df['target_category'] = df.apply(calc_target, axis=1)

Проверим остались ли у нас пропущенные значения в `target_category`. 

In [39]:
df[df.target_category.isna()]

,date,link,title,release,target_category
74,2015-12-29,/press/pr/?file=29122015_113403if2015-12-29T10...,О фиксированном времени публикации ключевой ин...,"\nБанк России, следуя наилучшим практикам расп...",NaN
81,2015-05-06,/press/pr/?file=06052015_163917if2015-05-06T16...,О публикации Обзора ключевых показателей некре...,Банк России начинает публиковать на официально...,NaN


У нас 2 таких наблюдения посмотрим на текст этих пресс-релизов.

In [41]:
indexes = df[df.target_category.isna()].index

for index in indexes:
    release = df.loc[index]
    print(release.date)
    print('-' * 80)
    for i in range(0, len(release.release), 80):
        
        print(release.release[i:i+80])

2015-12-29 00:00:00
--------------------------------------------------------------------------------

Банк России, следуя наилучшим практикам распространения данных, с 1 января 2016
 года вводит единый порядок публикации ключевой информации по статистике внешнег
о сектора — данные будут размещаться на официальном сайте Банка России в фиксиро
ванное время 16.00 мск. 
Даты публикации статистических данных, размещаемых в фи
ксированное время, содержатся в Календаре публикаций официальной статистической 
информации: 
http://www.cbr.ru/statistics/indcalendar/.
Новый порядок публикации
 обеспечит пользователям более удобные условия получения ключевой статистической
 информации и позволит эффективно планировать работу с этими данными.При использ
овании материала ссылка на Пресс-службу Банка России обязательна.

2015-05-06 00:00:00
--------------------------------------------------------------------------------
Банк России начинает публиковать на официальном сайте в информационно-телекоммун
ик

Эти релизы не связаны с изменением по ключевой ставке, удалим их из выборки.

In [46]:
df.dropna(subset=['target_category'], inplace=True)
df.reset_index(drop=True, inplace=True)

Теперь создадим переменную `interest_rate`, которая будет содержать величину процентной ставки.

In [50]:
def get_rate(row):
    m = re.search(r'до\s+(\d+,?\d*)%', row.release)
    if m is None or m.start(1) > 200:
        m = re.search(r'(\d+,?\d*)%', row.release)
    return float(m.group(1).replace(',', '.'))


df['interest_rate'] = df.apply(get_rate, axis=1)

Так как задачей нашей модели будет предсказание будущего решения по ключевой ставки, сделаем сдвиг `target_category` на один. И создадим новую переменную `target`, которая будет содержать изменение ключевой ставки на следующем заседании.   

In [56]:
df.target_category = df.target_category.shift(1)
df['target'] = df.interest_rate.shift(1) - df.interest_rate

In [57]:
df

,date,link,title,release,target_category,interest_rate,target
0,2024-09-13,/press/pr/?file=13092024_133000Key.htm,Банк России принял решение повысить ключевую с...,\n \nСовет директоров Банка России 13 сентября...,NaN,19.0,NaN
1,2024-07-26,/press/pr/?file=26072024_133000Key.htm,Банк России принял решение повысить ключевую с...,\n \nСовет директоров Банка России 26 июля 202...,NaN,18.0,1.0
2,2024-06-07,/press/pr/?file=07062024_133000Key.htm,Банк России принял решение сохранить ключевую ...,\n \nСовет директоров Банка России 7 июня 2024...,1.0,16.0,2.0
3,2024-04-26,/press/pr/?file=26042024_133000key.htm,Банк России принял решение сохранить ключевую ...,\n \nСовет директоров Банка России 26 апреля 2...,1.0,16.0,0.0
4,2024-03-22,/press/pr/?file=22032024_133000key.htm,Банк России принял решение сохранить ключевую ...,\n \nСовет директоров Банка России 22 марта 20...,0.0,16.0,0.0
...,...,...,...,...,...,...,...
91,2014-02-14,/press/pr/?file=14022014_133316stavka_+table.htm,О ключевой ставке Банка России,\nСовет директоров Банка России 14 февраля 201...,0.0,5.5,1.5
92,2013-12-13,/press/pr/?file=131213_133004stavka_table.htm,О ключевой ставке Банка России,\nСовет директоров Банка России 13 декабря 201...,1.0,5.5,0.0
93,2013-11-08,/press/pr/?file=131108_133008stavka_+table.htm,О ключевой ставке Банка России,\nСовет директоров Банка России 8 ноября 2013 ...,0.0,5.5,0.0
94,2013-10-14,/press/pr/?file=131014_133001refi_rate_tab.htm,О ключевой ставке Банка России,"\nПресс-служба Банка России сообщает, что Сове...",0.0,5.5,0.0
